In [8]:
import numpy as np
import pandas as pd
from utils import *
import random
import re
import keras.utils
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.models import load_model
from pickle import dump, load
import keras.backend as K
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def


In [9]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

# load the model
model = load_model('model.h5')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
text = np.random.randint(1, high=26, size=6, dtype='l')
remaining = np.array([0,0,0,0,0,0,0,0,0,0,0])
text = np.concatenate((text, remaining))
text  = text.reshape(1,17,1)

In [12]:
print (text)

[[[11]
  [11]
  [ 2]
  [21]
  [21]
  [17]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]]]


In [13]:
prediction = model.predict(text)

In [14]:
print (prediction.shape)

(1, 17, 27)


In [16]:
export_path = "export/main_model/5"

builder = tf.saved_model.builder.SavedModelBuilder(export_path)

print(model.input)
print(model.output)

Tensor("lstm_3_input:0", shape=(?, 17, 1), dtype=float32)
Tensor("dense_1/truediv:0", shape=(?, 17, 27), dtype=float32)


In [17]:
signature = predict_signature_def(inputs={"inputs": model.input},
                                  outputs={"outputs": model.output})


builder.add_meta_graph_and_variables(sess=sess,
                                    tags=[tag_constants.SERVING],
                                    signature_def_map={'predict': signature})
builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'export/main_model/5/saved_model.pb'


b'export/main_model/5/saved_model.pb'

[[[11]
  [11]
  [ 2]
  [21]
  [21]
  [17]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]]]


In [41]:
from oauth2client.client import GoogleCredentials
import googleapiclient.discovery
import numpy as np
import tensorflow as tf


text = np.random.randint(1, high=26, size=6, dtype='l')
remaining = np.array([0,0,0,0,0,0,0,0,0,0,0])
text = np.concatenate((text, remaining))
text  = text.reshape(1,17,1)

# print (text.tolist())

# # text_tf = tf.convert_to_tensor(text, np.float32)

# # sess = tf.InteractiveSession()
# # print(text_tf.eval())


# print (type([text]))
# Change this values to match your project
PROJECT_ID = "earningsprediction"
MODEL_NAME = "nameprediction1"
CREDENTIALS_FILE = "credentials.json"

# Connect to the Google Cloud-ML Service
credentials = GoogleCredentials.from_stream(CREDENTIALS_FILE)
service = googleapiclient.discovery.build('ml', 'v1', credentials=credentials)

# Connect to our Prediction Model
name = 'projects/{}/models/{}'.format(PROJECT_ID, MODEL_NAME)
response = service.projects().predict(
    name=name,
    body={'instances': text.tolist()}
).execute()

# Report any errors
if 'error' in response:
    raise RuntimeError(response['error'])

# Grab the results from the response object
results = response['predictions']

# Print the results!
print(results)


[{'outputs': [[8.395472104893997e-05, 3.252906935813371e-06, 2.295927879458759e-05, 0.000566889822948724, 0.0029274658299982548, 0.007536683697253466, 0.012193442322313786, 0.03666779026389122, 0.13215865194797516, 0.20363597571849823, 0.13836899399757385, 0.2263709157705307, 0.20678399503231049, 0.01468566618859768, 0.008891092613339424, 0.00783014390617609, 0.0003421048168092966, 0.0006140194600448012, 0.00011077459203079343, 0.00020158791448920965, 1.3031104799665627e-06, 2.035297029578942e-06, 1.373575742036337e-07, 2.1050370335728985e-08, 6.66212400801669e-08, 2.1319832566035757e-09, 1.848871846732436e-07], [1.7453617218166073e-08, 4.74924917170938e-07, 0.9990001320838928, 0.0009663217351771891, 3.193449447280727e-05, 1.3608513427243452e-07, 4.5436570417223265e-07, 4.4336485416351934e-07, 9.048778792930534e-08, 4.108866580310178e-09, 8.135798045189654e-16, 1.8128854537349803e-12, 2.917777998379112e-12, 1.0451625594718253e-13, 1.7600738485991482e-16, 2.0844537126588875e-24, 5.28923

In [42]:
print (len(results[0]['outputs']))

17


In [43]:
from pickle import dump, load
mapping = load(open('mapping.pkl', 'rb'))


In [44]:
print (mapping)

{' ': 0, ',': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'y': 25, 'z': 26}


In [45]:

predictedvalues = []



for values in results[0]['outputs']:
#     print (values)
#     print (values)
#     print (np.argmax(values))
    predictedvalues.append(np.argmax(values))


print (predictedvalues)


[11, 2, 2, 11, 10, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [46]:
predictedchars = []
for value in predictedvalues:
    predictedchars.append(list(mapping.keys())[list(mapping.values()).index(value)])
print (predictedchars)

['j', 'a', 'a', 'j', 'i', 'n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
